In [ ]:
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical



In [ ]:
# Detect device
device_name = tf.test.gpu_device_name()
if device_name:
    print(f"💻 GPU is available: {device_name}")
else:
    print("⚠️ GPU not found. Using CPU...")


💻 GPU is available: /device:GPU:0


In [ ]:
# Smaller CNN (4 conv layers)
def build_small_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
# Data loader and preprocessor
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical

def prepare_dataset(name):
    if name == "mnist":
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)

    elif name == "fashion_mnist":
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)

    elif name == "cifar10":
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    else:
        raise ValueError("Unknown dataset")

    # Normalize and one-hot encode
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    # Reduce data for faster CPU training
    x_train = x_train[:10000]
    y_train = y_train[:10000]

    return x_train, y_train, x_test, y_test, x_train.shape[1:]


In [ ]:
# Training and evaluation
import time

def train_model(dataset_name, epochs_list=[5, 10]):
    print(f"\n📦 Dataset: {dataset_name}")

    # Corrected line split
    x_train, y_train, x_test, y_test, input_shape = prepare_dataset(dataset_name)

    results = []

    for epochs in epochs_list:
        model = build_small_cnn_model(input_shape, 10)
        start = time.time()
        history = model.fit(
            x_train, y_train,
            epochs=epochs,
            batch_size=64,
            validation_split=0.1,
            verbose=0
        )
        end = time.time()

        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
        duration = end - start

        print(f"🧪 Epochs: {epochs} | Accuracy: {test_acc:.4f} | Time: {duration:.2f}s")
        results.append((epochs, test_acc, duration))

    return results


In [ ]:
# Run for each dataset
all_results = {}
datasets = ["mnist", "fashion_mnist"]

for name in datasets:
    all_results[name] = train_model(name)



📦 Dataset: mnist
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🧪 Epochs: 5 | Accuracy: 0.9748 | Time: 14.55s


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🧪 Epochs: 10 | Accuracy: 0.9830 | Time: 12.20s

📦 Dataset: fashion_mnist
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🧪 Epochs: 5 | Accuracy: 0.8630 | Time: 8.68s


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🧪 Epochs: 10 | Accuracy: 0.8774 | Time: 12.94s


In [ ]:
# Print final summary
print("\n✅ Summary (GPU Optimized):")
print("Dataset\t\tEpochs\tAccuracy\tTime (s)")

for dataset, records in all_results.items():
    for epochs, acc, dur in records:
        print(f"{dataset}\t{epochs}\t{acc:.4f}\t\t{dur:.2f}")



✅ Summary (GPU Optimized):
Dataset		Epochs	Accuracy	Time (s)
mnist	5	0.9748		14.55
mnist	10	0.9830		12.20
fashion_mnist	5	0.8630		8.68
fashion_mnist	10	0.8774		12.94
